In [1]:
import pandas as pd
# 定义数据集
df = pd.DataFrame({
    "name": ["A", "B", "C", "B", "C", "A", "C"],
    "value": [1, 3, 7, 2, 4, 3, 1]
})
# 我们是先生成了一个分组对象df.groupby("name")，然后再做了一次我们非常熟悉的索引，最后是进行了sum运算
df.groupby('name')['value'].sum()

name
A     4
B     5
C    12
Name: value, dtype: int64

In [3]:
# 我们对Groupby对象进行迭代
grouped = df.groupby('name')
grouped

In [5]:
for name, group in grouped:
    print(name)
    print(group)
    print('#' * 20)

A
  name  value
0    A      1
5    A      3
####################
B
  name  value
1    B      3
3    B      2
####################
C
  name  value
2    C      7
4    C      4
6    C      1
####################


In [6]:
trunk = dict(list(grouped))
trunk['A']

,name,value
0,A,1
5,A,3


In [8]:
# 通过实际的方式创建一个数据表
# 通过随机数的方式产生数据，数据包含班级、性别、以及5门课程的成绩，共200个学生的数据
import numpy as np
import pandas as pd
# 通过随机数生成介于50-99分之间的200×5的数据清单
frame = pd.DataFrame(np.random.randint(50, 100, size=(260, 5)),
                     columns='语文 数学 英语 音乐 体育'.split())
# 随机生成200名学生的性别
frame['性别'] = np.random.choice(['男', '女'], size=260, p=[0.53, 0.47])
# 随机生成200名学生的班级
frame['班级'] = np.random.choice(['一班', '二班', '三班', '四班'],
                               size=260,
                               p=[0.25, 0.26, 0.24, 0.25])
frame.head()

,语文,数学,英语,音乐,体育,性别,班级
0,53,51,57,88,94,女,三班
1,63,85,94,94,87,女,一班
2,93,61,82,93,97,男,三班
3,90,63,60,60,54,女,二班
4,76,82,92,94,72,男,四班


In [11]:
# 一次性查看每个分组下的数学和英语的最大值和最小值。这里推荐一个聚合运算方法aggregate
# 需要注意的是，对上文提到的常用统计方法，函数名是以字符串的形式传入的
frame.groupby(['班级', '性别'])[['数学', '英语']].agg(['max', 'min'])

数学      英语    
      max min max min
班级 性别                
一班 女   99  50  98  56
   男   97  50  99  54
三班 女   99  51  99  57
   男   99  50  99  52
二班 女   99  50  99  50
   男   99  50  98  50
四班 女   96  51  98  51
   男   98  50  97  51

In [12]:
# 需要注意的是，对上文提到的常用统计方法，函数名是以字符串的形式传入的
def max_gap(arr):
    return arr.max() - arr.min()


# 一次性查看全部科目的最高分、最低分、最大分差
# 注意：这里传入函数名，需要与字符串传递的方式区分开来
frame.groupby(['班级', '性别']).agg(['max', 'min', max_gap])

语文              数学              英语              音乐              体育      \
      max min max_gap max min max_gap max min max_gap max min max_gap max min   
班级 性别                                                                           
一班 女   98  52      46  99  50      49  98  56      42  98  52      46  97  50   
   男   99  50      49  97  50      47  99  54      45  99  50      49  96  50   
三班 女   99  50      49  99  51      48  99  57      42  99  51      48  98  50   
   男   98  51      47  99  50      49  99  52      47  98  53      45  98  50   
二班 女   99  50      49  99  50      49  99  50      49  99  52      47  98  51   
   男   99  50      49  99  50      49  98  50      48  99  51      48  99  51   
四班 女   99  50      49  96  51      45  98  51      47  95  53      42  99  52   
   男   98  52      46  98  50      48  97  51      46  99  51      48  99  52   

               
      max_gap  
班级 性别          
一班 女       47  
   男       46  
三班 女       48  
   男       48  
二班 女       47  
   男       48  
四班 女       47  
   男       47

In [13]:
# 比如对于数学，我们最关注的是最高分，对于音乐，我们最关注的是最大分差。这种情况下，在agg()方法中传入一个列名和方法组成的字典即可
frame.groupby(['班级', '性别']).agg({'数学': 'max', '音乐': max_gap})

数学  音乐
班级 性别        
一班 女   99  46
   男   97  49
三班 女   99  48
   男   99  45
二班 女   99  47
   男   99  48
四班 女   96  42
   男   98  48

In [15]:
# 定义top函数，可以查看xx科目的最高分，并返回最高分的学生的全部成绩
# n 表示选择TOP n
def top(grade, n=1, subject='数学'):
    return grade.sort_values(by=subject, ascending=False)[:n]


# 查看 数学最高分的学生的全部成绩
frame.groupby(['班级', '性别']).apply(top, 3, '数学')

语文  数学  英语  音乐  体育 性别  班级
班级 性别                               
一班 女  83   66  99  59  56  91  女  一班
      236  62  98  72  55  54  女  一班
      31   55  98  90  91  56  女  一班
   男  176  74  97  86  72  60  男  一班
      116  96  96  92  60  74  男  一班
      205  56  94  79  72  95  男  一班
三班 女  193  99  99  91  68  81  女  三班
      187  95  88  90  73  53  女  三班
      71   83  87  78  58  67  女  三班
   男  97   52  99  66  69  82  男  三班
      52   70  99  86  54  59  男  三班
      166  72  98  68  91  66  男  三班
二班 女  167  99  99  71  85  52  女  二班
      155  83  98  82  72  81  女  二班
      46   88  98  99  94  70  女  二班
   男  201  82  99  67  99  99  男  二班
      254  91  96  65  94  87  男  二班
      43   72  95  66  90  65  男  二班
四班 女  29   52  96  83  76  78  女  四班
      5    68  94  62  68  55  女  四班
      12   58  94  86  68  89  女  四班
   男  233  58  98  80  75  63  男  四班
      230  86  95  81  53  66  男  四班
      130  79  95  91  63  54  男  四班

In [21]:
# 更一般地，我们可以返回一个自定义的字典格式，来实现复杂的运算。例如我们希望同时查看数学的最高分、最低分、以及最大分差
def rule(group):
    _max = group.max()
    _min = group.min()
    return {'max': _max, 'min': _min, "gap": _max - _min}


# 注意，这里需要用unstak方法，把堆叠格式的数据转换成表格形式
frame.groupby(['班级', '性别'])['数学'].apply(rule).unstack()

max  min  gap
班级 性别               
一班 女    99   50   49
   男    97   50   47
三班 女    99   51   48
   男    99   50   49
二班 女    99   50   49
   男    99   50   49
四班 女    96   51   45
   男    98   50   48

In [22]:
# 例如我们前面在数据清洗环节讲过，可以用均值填充。那我们是否可以更加灵活，利用各个分组的均值去填充呢，答案是肯定的。
data = pd.DataFrame(np.random.randn(12), index=['A', 'B', "C"] * 4)
data.iloc[1] = np.nan
data.iloc[5] = np.nan
data.iloc[10] = np.nan
data

,0
A,-0.446004
B,NaN
C,-0.347175
A,-0.234247
B,-0.464243
C,NaN
A,-0.254409
B,-0.366511
C,0.848114
A,0.813340


In [23]:
# 按照分组的均值进行填充
# group_keys用来禁止显示分组键
data.groupby(data.index, group_keys=False).apply(lambda g: g.fillna(g.mean()))

,0
A,-0.446004
A,-0.234247
A,-0.254409
A,0.813340
B,-0.415377
B,-0.464243
B,-0.366511
B,-0.415377
C,-0.347175
C,-0.057795
